## Transfer Resting State Eyetracking Files to Compute Canada (.ps1)

VERSION: `Structural_Eyetrack_Auto_Sync_General.ps1`

### Advanced Explanation

This PowerShell script facilitates the transfer of directories created within the last 24 hours from the local system to a specified remote server path. It starts by obtaining the current date and prompting the user for their username. The user is then asked to verify or enter the remote path, constructed as `"/home/$username/scratch/RestingState_EyeTracking/"`.

Using `Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }`, the script identifies directories created in the last 24 hours. For each of these directories, it calculates the directory size in MB and prompts the user to confirm the transfer to the server. Upon confirmation, `scp` is used to securely copy the directory to the remote path. The script tracks and outputs the count of successfully transferred directories, along with their names. It concludes by waiting for the user to press Enter to close the script.

### User-Friendly Explanation

This script helps you back up your newest folders to a server. It first asks for your username and confirms the server path where the folders will go. Then, it looks for folders created in the last day and shows their sizes, asking if you want to back up each one to the server.

If you agree to back up a folder, the script securely transfers it to the server. It keeps track of how many folders it successfully transfers and lists their names at the end. Finally, the script waits for you to press Enter before it closes. This is a useful tool for regularly backing up new folders to a secure server.

### Oversimplified Explanation

This should be kept and run where resting state eyetracking session folders are backed up. It will send new ones from the local device to your scratch folder on Compute Canada. 

### Note
    
Track meta data before sending and compare it to the output of the check scripts. 

In [ ]:
$currentDate = Get-Date

$username = Read-Host "Please enter your username"

$remotePathBase = "/home/$username/scratch/RestingState_EyeTracking/"

Write-Output "The constructed remote path is: $remotePathBase"
$pathConfirmation = Read-Host "Is this path correct? [Y/N]"

if ($pathConfirmation -ne "Y") {
    $remotePathBase = Read-Host "Please enter the full correct remote path"
}

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name

    $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
    $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($directorySize) MB] to the server at `"$remotePath`"? [Y/N]"

    if ($userInput -eq "Y") {
        scp -r $dir.FullName "$username@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            
            Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}

Read-Host -Prompt "Press Enter to Close"

VERSION: `Structural_Eyetrack_Auto_Sync_(name).ps1`

Replace remote pathbase and scp command to reflect the user and save with your name using Powershell ISE. Note that there will be no prompts as the script is designed for maximum efficiency so make sure you know what you are doing. 

In [ ]:
$currentDate = Get-Date

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/FinalRestingStateEyetracking/"

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name

    scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
    if ($?) {
        $transferredCount++
        $transferredDirectories += $dir.Name
        
        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
    } else {
        Write-Warning "Failed to transfer directory: $dir"
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}
